In [1]:
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk

/Users/olgacravesana/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# dictionary from excel file

In [2]:
excell = pd.read_excel('PWC.xlsx', sheet_name = 'Resume' )

In [3]:
# excell.head(59)

In [4]:
# excell.iloc[28,:]

In [5]:
commission = {'abanca': {'1':'Emissão de extrato','2':'Fotocópias de segundas vias de talões de depósito',\
                        '3': 'Manutenção de conta', '4': {'Levantamento de numerário':{'4.1':'Ao balcão, com apresentação de cheque',\
                                                                                      '4.2': 'Ao balcão, sem apresentação de cheque'},\
                                                         }, '5': 'Adesão ao serviço de banca à distância', '6':'Depósito de moedas metálicas',\
                        '7':'Alteração de titulares'}, \
              'ctt':{'1':'Extrato Integrado mensal', '2':'Consulta de Saldo de conta DO com comprovativo',\
                     '3':'Consulta de Movimentos de conta DO com comprovativo', '4': 'Emissão 2as Vias de Avisos e Outros Documentos',\
                     '5':'Alteração de titularidade'},
              'bic':{'1':'Manutenção de conta','2':'Levantamento de numerário','3':'Levantamento USD em contas USD',\
                    '4': 'Extracto integrado','5':'Extracto avulso'}}

# extract pdf

In [6]:
def len_sentences(x):
    if len(nltk.sent_tokenize(x))== 1:
        return x.replace('\n','. ')
    return x.replace('\n','')

In [7]:
def extract_clean(pdf,n_page):
    file = pdfplumber.open(pdf)
    page = file.pages[n_page]
    text = page.extract_text().lower()
    text = re.sub('isento', '0,00', text)
    text = re.sub('n/a', str(np.nan), text)
    text = re.sub('--', str(np.nan), text)
    text=text.replace('\n','')
    text = len_sentences(text)
    text = nltk.sent_tokenize(text)
    return text

In [8]:
abanca = extract_clean('data/ABANCA2021com.pdf', 4)
ctt = extract_clean('data/BancoCTT2021.pdf', 4)
bai = extract_clean('data/bancoBAIeurope2021com.pdf', 4)
bankinter = extract_clean('data/bankinter.pdf', 4)

# search for commisions 

In [9]:
abanca[:3]
# ctt
# bai
# bankinter

['entrada em vigor: 20-nov-2020  1.  contas de depósito (particulares)                                                                                   índice  1.1 depósitos à ordem  comissões  acresce  outras   euros (mín/máx)  valor anual  imposto  condições produtos: fora de comercialização: conta d.o.',
 'particulares; conta ordenado; conta standard; conta future; conta kids; conta global em comercialização: conta base; conta private; conta value; conta smart; conta futuro; conta serviços mínimos bancários, conta moeda estrangeira e conta abanca internacional.',
 'as comissões abaixo indicadas ocorrem no momento em que se verifica a operação e aplicam-se às contas acima identificadas, salvo nos casos em que expressamente se exclui.']

In [20]:
def search_com(x, pdf, page):
    decod = {}
    if x not in commission:
        return 'not in the dictionary'
    decod = commission[x]
    file = extract_clean(pdf, page)
    return file